In [171]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from _spotify.spotipy_auth import spotipy_auth

spotipy_auth()

sp = Spotify(auth_manager=SpotifyClientCredentials())
sp.search('forest fire', type = 'track')['tracks']['items'][0]['id']

'6Vejmu5dz6MMOnn2pIko9q'